# This notebook takes the *_second_codings.csv ground truth files generated from makeSecBySec.R and adds an extra column with the fixed coding scheme
## The fixed coding scheme generalizes postures more and joins some specific postures together

In [1]:
import pandas as pd
import numpy as np
import os

## Get ground truth files ("*_second_codings.csv")

In [5]:
files = os.listdir("GroundTruth_old")
files.sort()
files

['AM01DO1_J_FINAL_R_second_codings.csv',
 'AM01DO2_M_FINAL_R_second_codings.csv',
 'AM02DO1_J_FINAL_R_second_codings.csv',
 'AM02DO2_J_copyA_FINAL_R_second_codings.csv',
 'AM03DO1_M_FINAL_R_second_codings.csv',
 'AM03DO2_J_FINAL_R_second_codings.csv',
 'AM04DO1_J_FINAL_R_second_codings.csv',
 'AM04DO2_M_FINAL_R_second_codings.csv',
 'AM05DO2re_R_FINAL_C_second_codings.csv',
 'AM06DO1_J_FINAL_C_second_codings.csv',
 'AM06DO2_N_FINAL_C_second_codings.csv',
 'AM07DO1_R_FINAL_C_second_codings.csv',
 'AM07DO2_M_FINAL_R_second_codings.csv',
 'AM08DO1_J_copyA_FINAL_R_second_codings.csv',
 'AM08DO1_J_copyB_FINAL_R_second_codings.csv',
 'AM08DO2_J_FINAL_R_second_codings.csv',
 'AM09DO1_N_FINAL_R_second_codings.csv',
 'AM09DO2_M_FINAL_R_second_codings.csv',
 'AM10DO2_M_FINAL_R_second_codings.csv',
 'AM11DO1_N_copyA_FINAL_C_second_codings.csv',
 'AM11DO2_R_copyA_FINAL_C_second_codings.csv',
 'AM11DO2_R_copyB_FINAL_C_second_codings.csv',
 'AM12DO1_J_FINAL_C_second_codings.csv',
 'AM12DO2_J_copyA_F

## Fix coding scheme; add 'posture_coding' column with fixed coding

In [39]:
for file in files:
    ground_truth = pd.read_csv("GroundTruth_old/" + file)
    ground_truth = ground_truth[:-1]

    # absorb all types of "LA - stand and move" together
    la = (ground_truth['primary_posture'] == 'LA- stand and move') | (ground_truth['primary_posture'] == 'LA- stand and move with upper body movement') | (ground_truth['primary_posture'] == 'LA- stand and move with unidentifiable upper body movement')

    # code "LA - stand and move" as light/moderate/vigorous
    la_light = la & (ground_truth['primary_intensity'] == 'light')
    la_mod = la & (ground_truth['primary_intensity'] == 'moderate')
    la_vig = la & (ground_truth['primary_intensity'] == 'vigorous')

    # absorb walk and walk w/ load
    wa = (ground_truth['primary_posture'] == 'WA- walk') | (ground_truth['primary_posture'] == 'WA-walk with load')

    # code walking as light/moderate/vigorous
    wa_light = wa & (ground_truth['primary_intensity'] == 'light')
    wa_mod = wa & (ground_truth['primary_intensity'] == 'moderate')
    wa_vig = wa & (ground_truth['primary_intensity'] == 'vigorous')

    # absorb "LA - kneeling/squatting" and "LA - stretching"
    kneel_squat_stretch = (ground_truth['primary_posture'] == 'LA- kneeling/ squatting') | (ground_truth['primary_posture'] == 'LA- stretching')

    # absorb "LA - kneeling/squatting"/"LA-stretching" into "SB-sitting" if sedentary
    sed_kneel_squat_stretch = kneel_squat_stretch & (ground_truth['primary_intensity'] == 'sedentary')


    ground_truth['posture_coding'] = np.where(la_light, 'LA- stand and move light',
                                        np.where(la_mod, 'LA- stand and move moderate',
                                            np.where(la_vig, 'LA- stand and move vigorous',
                                                np.where(wa_light, 'WA- walk light',
                                                    np.where(wa_mod, 'WA- walk moderate', 
                                                        np.where(wa_vig, 'WA- walk vigorous',
                                                            np.where(sed_kneel_squat_stretch, 'SB-sitting',
                                                                ground_truth['primary_posture'])))))))
    
    ground_truth.to_csv(file)
    print("updated " + file)

updated AM02DO1_J_FINAL_R_second_codings.csv
updated AM19DO2_R_FINAL_C_second_codings.csv
updated AM27DO1_R_FINAL_C_second_codings.csv
updated AM10DO2_M_FINAL_R_second_codings.csv
updated AM26DO2_R_copyB_FINAL_C_second_codings.csv
updated AM11DO2_R_copyB_FINAL_C_second_codings.csv
updated AM07DO2_M_FINAL_R_second_codings.csv
updated AM06DO2_N_FINAL_C_second_codings.csv
updated AM22DO1_R_FINAL_C_second_codings.csv
updated AM11DO1_N_copyA_FINAL_C_second_codings.csv
updated AM08DO1_J_copyA_FINAL_R_second_codings.csv
updated AM22DO2_R_FINAL_C_second_codings.csv
updated AM27DO2_R_FINAL_C_second_codings.csv
updated AM05DO2re_R_FINAL_C_second_codings.csv
updated AM09DO2_M_FINAL_R_second_codings.csv
updated AM16DO1_M_FINAL_R_second_codings.csv
updated AM12DO2_J_copyB_FINAL_C_second_codings.csv
updated AM24DO2_R_FINAL_C_second_codings.csv
updated AM01DO2_M_FINAL_R_second_codings.csv
updated AM01DO1_J_FINAL_R_second_codings.csv
updated AM13DO2_M_FINAL_R_second_codings.csv
updated AM24DO1_R_FINAL

## Check posture class labels

In [2]:
def build_giant_ground_truth(gtdir):
    giant_ground_truth = pd.DataFrame()
    size_check = 0
    
    files = os.listdir(gtdir)
    
    for file in files:
        ground_truth = pd.read_csv(gtdir + file)
        giant_ground_truth = giant_ground_truth.append(ground_truth)
        
    return giant_ground_truth

In [43]:
new_giant_ground_truth = build_giant_ground_truth("GroundTruth_new/")

In [44]:
set(new_giant_ground_truth['posture_coding'].values)

{'LA- stand',
 'LA- stand and move light',
 'LA- stand and move moderate',
 'LA- stand and move vigorous',
 'SB- lying',
 'SB-sitting',
 'SP- bike',
 'SP- other sport movement',
 'WA- ascend stairs',
 'WA- descend stairs',
 'WA- running',
 'WA- walk light',
 'WA- walk moderate',
 'WA- walk vigorous',
 'private/not coded'}